In [5]:
import sys
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pyltp import Segmentor 
import warnings
sys.setrecursionlimit(10000)
warnings.filterwarnings("ignore")

In [ ]:
# #去除说文的双引号---------------可不用
# import re
# filename = r'C:\Users\Administrator\Desktop\sentence.txt'
# with open(filename, 'rt', encoding='utf-8') as f:
#     with open(r'C:\Users\Administrator\Desktop\sentence1.txt','wt',encoding='utf-8') as w:
#         text = re.sub(r'“|”','',f.read())
#         for line in text.split('\n'):
#             w.write(line+'\n')

In [6]:
filename_path = r'sentence1.txt'#挑选出的人名+说文

In [9]:
text = pd.read_csv(filename_path, delimiter='\t',names=['人名','文字'])

In [10]:
def cut(string):
    return ' '.join(segmentor.segment(string))

In [11]:
#分词
segmentor = Segmentor()
segmentor.load(r'/home/aistudio/data/data11885/cws.model')
text_list = text['文字'].tolist()
text_process = [cut(tl) for tl in text_list]
segmentor.release()

In [12]:
vectorizer = CountVectorizer(ngram_range=(1, 2),token_pattern=r'\b\w+\b')#小心max_df=1
transformer = TfidfTransformer(sublinear_tf=True)
tfidf = transformer.fit_transform(vectorizer.fit_transform(text_process))

In [9]:
#自定义cosine距离
# def cos_distance(v1,v2):
#     '''
#     params:v1   输入的观点
#     params:v2   已有观点集合的某个观点、
#     return:     余弦距离（相似度）
#     '''
#     length = np.linalg.norm(v1)*np.linalg.norm(v2)
#     if length != 0:
#         cos_theta = np.dot(v1,v2)/length
#         return cos_theta
#     else:
#         return 0

In [13]:
#自定义cosine矩阵
# cos_vector = np.full_like(tfidf.toarray(),-1)#初始化矩阵
# for i,tfidf_one in enumerate(tfidf_float):
#     if i%10000==0:print(i)
#     for j,tfidf_two in enumerate(tfidf.toarray()[i+1:]):        
#         similarity= cos_distance(tfidf_one,tfidf_two)
#         print(similarity)
#         cos_vector[i,i+j+1] = similarity 

tfidf_float = np.around(tfidf.toarray(),decimals=3)
cosine_vector = cosine_similarity(tfidf_float)
cosine_vector = np.tril(cosine_vector,-1)

In [14]:
cluster_list = []#写成类，可以定义成初始属性

In [15]:
def find_2D_ndaray(vectors,one_data):
    for x,vector in enumerate(vectors):
        for y,v in enumerate(vector):
            if one_data == v:
                return x,y
    else:
        return False

In [21]:
#threshold要调试。如果取0.5就只有一类
def cluster_new(cosine_vector,threshold=0.0037,count1=0):
    
    if np.max(cosine_vector) == -1: return cluster_list#如果只剩下第0条 则结束
    max_shreshold = np.max(cosine_vector)
    max_shreshold_index = np.unravel_index(np.argmax(cosine_vector),cosine_vector.shape)#只返回一个（x,y）,shape不规则，只输出第一个shape
    if max_shreshold >= threshold:
        
        if not cluster_list:#第一次为空，直接添加
            cluster_list.append(list(max_shreshold_index))
            
        else:#第二次按条件添加，如果已经在列表中，则添加到相应的子列表中，如果不在，则新增类别，加入列表中
            data_index = find_2D_ndaray(cluster_list,max_shreshold_index[1])
            
            if data_index:
               
                cluster_list[data_index[0]].append(max_shreshold_index[0])
                
            else:
                cluster_list.append(list(max_shreshold_index))
    else:
        cluster_list.append([max_shreshold_index[0]])
    cosine_vector[max_shreshold_index[0],:] = -1 #已添加的行变为-1
    
    cluster_new(cosine_vector)

In [17]:
cluster_text_index = cluster_new(cosine_vector)#二维数组

## 提取类别文本的两种方法，任选其一

In [ ]:
# #1、在Dataframe添加标签，再分组。可作类中一方法
# text['label'] = None
# for cluster_i, t in enumerate(cluscluster_text_index):
#     text.loc[t,'label'] = cluster_i 

In [20]:
#2、直接根据类别提取文本
cluster_text = [text.loc[t,'文字'].tolist() for t in cluster_list]

 ...]